# Parsing all the Data from the Files available on "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University"  

Link : https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

In [640]:
import pandas as pd
import os
import ast
import numpy as np
from datetime import datetime

In [641]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Raw Data (CSSE) at Johns Hopkins University\\COVID-19\\csse_covid_19_data\\csse_covid_19_daily_reports\\"

In [642]:
df_final = pd.DataFrame({'Province_State' : [], 'Country_Region' : [], 'Last_Update' : [], 'Lat' : [], 'Long_' : [],'Confirmed' : [], 'Deaths' : [], 'Recovered' : [], 'Active' : [], 'Combined_Key' : [],'Incident_Rate' : [], 'Case_Fatality_Ratio' : []})
for file in os.listdir(path):
    if file.endswith(".csv"):
        df = pd.read_csv(path + file, index_col=False)
        try:
            df_temp = df[df['Country_Region'] == "Pakistan"].copy()
            df_temp.drop(['FIPS', 'Admin2'], axis = 1, inplace = True)
            
            if(len(df_temp.columns) == 12):
                df_final = df_final.append(df_temp, True)
                df_final.drop(['Combined_Key'], axis = 1, inplace = True)
               
        except :
            continue
            

In [643]:
df_final.isnull().sum()

Province_State           12
Country_Region            0
Last_Update               0
Lat                       0
Long_                     0
Confirmed                 0
Deaths                    0
Recovered              1323
Active                 1323
Incident_Rate          1076
Case_Fatality_Ratio    1076
Incidence_Rate         3206
Case-Fatality_Ratio    3206
dtype: int64

In [644]:
df_final.drop(['Active'], axis = 1, inplace = True)
df_final = df_final.dropna(subset=['Province_State'])

In [645]:
df_final.drop(['Incident_Rate', 'Incidence_Rate', 'Case-Fatality_Ratio', 'Case_Fatality_Ratio'], axis = 1, inplace = True)

In [646]:
df_final.isnull().sum()

Province_State       0
Country_Region       0
Last_Update          0
Lat                  0
Long_                0
Confirmed            0
Deaths               0
Recovered         1323
dtype: int64

In [647]:
# df_temp = df_final['Case_Fatality_Ratio'].fillna(df_final['Case-Fatality_Ratio'])
# df_final.loc[df_final['Case_Fatality_Ratio'].isnull(), 'Case_Fatality_Ratio'] = df_temp

In [648]:
# df_temp = df_final['Incident_Rate'].fillna(df_final['Incidence_Rate'])
# df_final.loc[df_final['Incident_Rate'].isnull(), 'Incident_Rate'] = df_temp

In [649]:
# df_final = df_final.dropna(subset=['Incidence_Rate', 'Case-Fatality_Ratio'])

In [650]:
df_final.isnull().sum()

Province_State       0
Country_Region       0
Last_Update          0
Lat                  0
Long_                0
Confirmed            0
Deaths               0
Recovered         1323
dtype: int64

In [651]:
len(df_final)

4270

In [652]:
df_final.to_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\JHU Data\\All_Province (06-2020 to 02-2022).csv", index = False)

# Remaining Data Preparation For Merging

In [653]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Raw Data (CSSE) at Johns Hopkins University\\COVID-19\\remaining_province_wise_data\\Pakistan_Province_summary_covid19_confirmed.csv"
df_confirmed = pd.read_csv(path, index_col=False)
df_confirmed.drop(['code'], axis = 1, inplace = True)
df_confirmed.sort_values(by=['Date'])
df_confirmed['Date'] = pd.to_datetime(df_confirmed['Date'])

confirmed_cases = df_confirmed[df_confirmed['Date'] < "2020-06-11"]
confirmed_cases.head()

,Province,Date,Confirmed cases
0,Azad Jammu and Kashmir,2020-03-10,0
1,Balochistan,2020-03-10,1
2,Gilgit-Baltistan,2020-03-10,2
3,Islamabad,2020-03-10,2
4,Khyber Pakhtunkhwa,2020-03-10,0


In [654]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Raw Data (CSSE) at Johns Hopkins University\\COVID-19\\remaining_province_wise_data\\Pakistan_Province_summary_covid19_death.csv"
df_deaths = pd.read_csv(path, index_col=False)
df_deaths.drop(['code'], axis = 1, inplace = True)
df_deaths.sort_values(by=['Date'])
df_deaths['Date'] = pd.to_datetime(df_deaths['Date'])

deaths = df_deaths[df_deaths['Date'] < "2020-06-11"]
deaths.head()

,Province,Date,Deaths
0,Azad Jammu and Kashmir,2020-03-10,0
1,Balochistan,2020-03-10,0
2,Gilgit-Baltistan,2020-03-10,0
3,Islamabad,2020-03-10,0
4,Khyber Pakhtunkhwa,2020-03-10,0


In [655]:
remaining_data = confirmed_cases.merge(deaths, how='inner', on=['Province', 'Date'])
remaining_data = remaining_data.rename(columns={"Confirmed cases" : "Confirmed"})
remaining_data['Date'] = pd.to_datetime(remaining_data['Date']).dt.date
remaining_data.head()

,Province,Date,Confirmed,Deaths
0,Azad Jammu and Kashmir,2020-03-10,0,0
1,Balochistan,2020-03-10,1,0
2,Gilgit-Baltistan,2020-03-10,2,0
3,Islamabad,2020-03-10,2,0
4,Khyber Pakhtunkhwa,2020-03-10,0,0


# Merging All the Data into One File

In [656]:
df_final['Last_Update'] = pd.to_datetime(df_final['Last_Update']).dt.date


In [657]:
df_final = df_final.rename(columns={"Province_State" : "Province","Last_Update" : "Date"})

In [658]:
len(remaining_data)

651

In [659]:
df_final.tail(7)

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered
4275,Azad Jammu and Kashmir,Pakistan,2022-01-01,34.027401,73.947253,34662.0,746.0,NaN
4276,Balochistan,Pakistan,2022-01-01,28.328492,65.898403,33638.0,364.0,NaN
4277,Gilgit-Baltistan,Pakistan,2022-01-01,35.792146,74.982138,10429.0,186.0,NaN
4278,Islamabad,Pakistan,2022-01-01,33.665087,73.121219,108666.0,967.0,NaN
4279,Khyber Pakhtunkhwa,Pakistan,2022-01-01,34.485332,72.091690,181402.0,5930.0,NaN
4280,Punjab,Pakistan,2022-01-01,30.811346,72.139132,445107.0,13070.0,NaN
4281,Sindh,Pakistan,2022-01-01,26.009446,68.776807,482029.0,7670.0,NaN


In [660]:
 df_final = df_final.append(remaining_data)

In [661]:
df_final.isnull().sum()

Province             0
Country_Region     651
Date                 0
Lat                651
Long_              651
Confirmed            0
Deaths               0
Recovered         1974
dtype: int64

In [662]:
df_final.loc[df_final['Country_Region'].isnull(), 'Country_Region'] = "Pakistan"

In [663]:
# df_final[df_final['Province'] == 'Sindh'][['Lat', 'Long_']]
# 26.009446, 68.776807
# df_final[df_final['Province'] == 'Punjab'][['Lat', 'Long_']]
# 30.811346, 72.139132
# df_final[df_final['Province'] == 'Gilgit-Baltistan'][['Lat', 'Long_']]
# 35.792146, 74.982138
# df_final[df_final['Province'] == 'Islamabad'][['Lat', 'Long_']]
# 33.665087, 73.121219
# df_final[df_final['Province'] == 'Khyber Pakhtunkhwa'][['Lat', 'Long_']]
# 34.485332, 72.09169
# df_final[df_final['Province'] == 'Balochistan'][['Lat', 'Long_']]
# 28.328492, 65.898403
# df_final[df_final['Province'] == 'Azad Jammu and Kashmir'][['Lat', 'Long_']]
# 34.027401, 73.947253

In [664]:
df_final.loc[df_final['Province'] == 'Sindh', 'Lat'] = 26.009446
df_final.loc[df_final['Province'] == 'Sindh', 'Long_'] = 68.776807

df_final.loc[df_final['Province'] == 'Punjab', 'Lat'] = 30.811346
df_final.loc[df_final['Province'] == 'Punjab', 'Long_'] = 72.139132

df_final.loc[df_final['Province'] == 'Gilgit-Baltistan', 'Lat'] = 35.792146
df_final.loc[df_final['Province'] == 'Gilgit-Baltistan', 'Long_'] = 74.982138

df_final.loc[df_final['Province'] == 'Islamabad', 'Lat'] = 33.665087
df_final.loc[df_final['Province'] == 'Islamabad', 'Long_'] = 73.121219

df_final.loc[df_final['Province'] == 'Khyber Pakhtunkhwa', 'Lat'] = 34.485332
df_final.loc[df_final['Province'] == 'Khyber Pakhtunkhwa', 'Long_'] = 72.09169

df_final.loc[df_final['Province'] == 'Balochistan', 'Lat'] = 28.328492
df_final.loc[df_final['Province'] == 'Balochistan', 'Long_'] = 65.898403

df_final.loc[df_final['Province'] == 'Azad Jammu and Kashmir', 'Lat'] = 34.027401
df_final.loc[df_final['Province'] == 'Azad Jammu and Kashmir', 'Long_'] = 73.947253

In [665]:
df_final.isnull().sum()

Province             0
Country_Region       0
Date                 0
Lat                  0
Long_                0
Confirmed            0
Deaths               0
Recovered         1974
dtype: int64

In [666]:
df_final.head()

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered
0,Azad Jammu and Kashmir,Pakistan,2021-01-02,34.027401,73.947253,8303.0,224.0,7691.0
1,Balochistan,Pakistan,2021-01-02,28.328492,65.898403,18181.0,184.0,17752.0
2,Gilgit-Baltistan,Pakistan,2021-01-02,35.792146,74.982138,4862.0,101.0,4713.0
3,Islamabad,Pakistan,2021-01-02,33.665087,73.121219,38020.0,421.0,34617.0
4,Khyber Pakhtunkhwa,Pakistan,2021-01-02,34.485332,72.091690,59023.0,1661.0,53942.0


In [667]:
df_final.to_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\JHU Data\\All_Province (03-2020 to 02-2022).csv", index = False)

# Working on Data From IHME

In [668]:
path2020 = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\data_download_file_reference_2020.csv"
path2021 = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\data_download_file_reference_2021.csv"
path2022 = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\data_download_file_reference_2022.csv"

In [669]:
df2020 = pd.read_csv(path2020, index_col = False)
df2021 = pd.read_csv(path2021, index_col = False)
df2022 = pd.read_csv(path2022, index_col = False)

In [670]:
dfNull2020 = df2020.columns[df2020.isnull().any()]
df2020.drop(dfNull2020, axis = 1, inplace = True)

dfNull2021 = df2021.columns[df2021.isnull().any()]
df2021.drop(dfNull2021, axis = 1, inplace = True)

dfNull2022 = df2022.columns[df2022.isnull().any()]
df2022.drop(dfNull2022, axis = 1, inplace = True)

In [671]:
df_All_IHME = pd.DataFrame()

provinces = ['Sindh', 'Punjab', 'Balochistan', 'Azad Jammu & Kashmir', "Gilgit-Baltistan", "Islamabad Capital Territory", "Khyber Pakhtunkhwa"]

for i in range(len(provinces)):
    df_All_IHME = df_All_IHME.append(df2020[df2020['location_name'] == provinces[i]])
    df_All_IHME = df_All_IHME.append(df2021[df2021['location_name'] == provinces[i]])
    df_All_IHME = df_All_IHME.append(df2022[df2022['location_name'] == provinces[i]])

In [672]:
len(df_All_IHME)

5943

In [673]:
df_All_IHME.head()

,location_id,date,version_name,location_name,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,seir_cumulative_upper,...,admis_lower_unvax,testing_mean,testing_lower,testing_upper,cumulative_all_vaccinated,cumulative_all_fully_vaccinated,cumulative_all_effectively_vaccinated,infection_fatality,infection_detection,infection_hospitalization
102920,53621,2020-02-04,reference,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102921,53621,2020-02-05,reference,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102922,53621,2020-02-06,reference,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102923,53621,2020-02-07,reference,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102924,53621,2020-02-08,reference,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [674]:
df_All_IHME.drop([df_All_IHME.columns[0], "location_id", "version_name"], axis = 1, inplace = True)

In [675]:
len(df_All_IHME)

5943

In [676]:
nan_value = float("NaN")
df_All_IHME.replace("", nan_value, inplace=True)
df_All_IHME.dropna(how='any', axis=1, inplace=True)

In [677]:
df_All_IHME.isnull().sum()

date                              0
location_name                     0
inf_mean                          0
inf_upper                         0
inf_lower                         0
seir_cumulative_mean              0
seir_cumulative_lower             0
seir_cumulative_upper             0
inf_cuml_mean                     0
inf_cuml_lower                    0
inf_cuml_upper                    0
seir_daily_mean                   0
seir_daily_lower                  0
seir_daily_upper                  0
seir_daily_unscaled_mean          0
seir_daily_unscaled_lower         0
seir_daily_unscaled_upper         0
seir_cumulative_unscaled_mean     0
seir_cumulative_unscaled_lower    0
seir_cumulative_unscaled_upper    0
cases_mean                        0
cases_lower                       0
cases_upper                       0
inf_cuml_mean_vax                 0
inf_cuml_lower_vax                0
inf_cuml_upper_vax                0
inf_cuml_mean_unvax               0
inf_cuml_lower_unvax        

In [678]:
df_All_IHME = df_All_IHME.rename(columns={'date' : 'Date', 'location_name' : 'Province'})

In [679]:
df_All_IHME['Date'] = pd.to_datetime(df_All_IHME['Date']).dt.date

In [680]:
df_All_IHME.head(7)

,Date,Province,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,seir_cumulative_upper,inf_cuml_mean,inf_cuml_lower,...,icu_beds_lower,admis_mean,admis_upper,admis_lower,admis_mean_vax,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax
102920,2020-02-04,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102921,2020-02-05,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102922,2020-02-06,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102923,2020-02-07,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102924,2020-02-08,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102925,2020-02-09,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102926,2020-02-10,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [681]:
df_All_IHME.loc[df_All_IHME['Province'] == "Azad Jammu & Kashmir", 'Province'] = "Azad Jammu and Kashmir"
df_All_IHME.loc[df_All_IHME['Province'] == "Islamabad Capital Territory", 'Province'] = "Islamabad"

In [682]:
df_All_IHME.head(7)

,Date,Province,inf_mean,inf_upper,inf_lower,seir_cumulative_mean,seir_cumulative_lower,seir_cumulative_upper,inf_cuml_mean,inf_cuml_lower,...,icu_beds_lower,admis_mean,admis_upper,admis_lower,admis_mean_vax,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax
102920,2020-02-04,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102921,2020-02-05,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102922,2020-02-06,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102923,2020-02-07,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102924,2020-02-08,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102925,2020-02-09,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102926,2020-02-10,Sindh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [683]:
df_All_IHME.to_csv("C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\IHME Data\\IHME.csv", index = False)

# Merging All the Data from JHU and IHME

In [684]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\JHU Data\\All_Province (03-2020 to 02-2022).csv"

In [685]:
df_JHU = pd.read_csv(path)
df_JHU.head(7)

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered
0,Azad Jammu and Kashmir,Pakistan,2021-01-02,34.027401,73.947253,8303.0,224.0,7691.0
1,Balochistan,Pakistan,2021-01-02,28.328492,65.898403,18181.0,184.0,17752.0
2,Gilgit-Baltistan,Pakistan,2021-01-02,35.792146,74.982138,4862.0,101.0,4713.0
3,Islamabad,Pakistan,2021-01-02,33.665087,73.121219,38020.0,421.0,34617.0
4,Khyber Pakhtunkhwa,Pakistan,2021-01-02,34.485332,72.091690,59023.0,1661.0,53942.0
5,Punjab,Pakistan,2021-01-02,30.811346,72.139132,139341.0,4085.0,124125.0
6,Sindh,Pakistan,2021-01-02,26.009446,68.776807,216632.0,3582.0,196134.0


In [686]:
df_JHU['Date'] = pd.to_datetime(df_JHU['Date']).dt.date

In [687]:
df_JHU_IHME = pd.merge(left=df_JHU, right=df_All_IHME, how='left', left_on=['Date', 'Province'], right_on=['Date', 'Province'])

In [688]:
df_JHU_IHME.head() 

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,inf_mean,inf_upper,...,icu_beds_lower,admis_mean,admis_upper,admis_lower,admis_mean_vax,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax
0,Azad Jammu and Kashmir,Pakistan,2021-01-02,34.027401,73.947253,8303.0,224.0,7691.0,3808.83,6326.40,...,43.54,44.32,87.53,18.54,0.0,0.0,0.0,44.315680,87.531220,18.540050
1,Balochistan,Pakistan,2021-01-02,28.328492,65.898403,18181.0,184.0,17752.0,7796.48,10691.60,...,45.43,51.53,99.96,24.12,0.0,0.0,0.0,51.534680,99.960110,24.121890
2,Gilgit-Baltistan,Pakistan,2021-01-02,35.792146,74.982138,4862.0,101.0,4713.0,141.94,291.55,...,3.48,2.40,5.13,1.03,0.0,0.0,0.0,2.395434,5.133114,1.029408
3,Islamabad,Pakistan,2021-01-02,33.665087,73.121219,38020.0,421.0,34617.0,1909.63,3303.93,...,24.67,26.61,43.25,11.48,0.0,0.0,0.0,26.606190,43.253210,11.482480
4,Khyber Pakhtunkhwa,Pakistan,2021-01-02,34.485332,72.091690,59023.0,1661.0,53942.0,73123.81,99366.35,...,463.77,466.71,1027.98,171.97,0.0,0.0,0.0,466.710000,1027.979000,171.967100


In [689]:
df_JHU_IHME.isnull().sum()

Province             0
Country_Region       0
Date                 0
Lat                  0
Long_                0
                    ..
admis_upper_vax      0
admis_lower_vax      0
admis_mean_unvax     0
admis_upper_unvax    0
admis_lower_unvax    0
Length: 64, dtype: int64

In [690]:
df_JHU_IHME.rename(columns={'Unnamed: 0':'S.No'}, inplace=True )

In [691]:
df_JHU_IHME.head()

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,inf_mean,inf_upper,...,icu_beds_lower,admis_mean,admis_upper,admis_lower,admis_mean_vax,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax
0,Azad Jammu and Kashmir,Pakistan,2021-01-02,34.027401,73.947253,8303.0,224.0,7691.0,3808.83,6326.40,...,43.54,44.32,87.53,18.54,0.0,0.0,0.0,44.315680,87.531220,18.540050
1,Balochistan,Pakistan,2021-01-02,28.328492,65.898403,18181.0,184.0,17752.0,7796.48,10691.60,...,45.43,51.53,99.96,24.12,0.0,0.0,0.0,51.534680,99.960110,24.121890
2,Gilgit-Baltistan,Pakistan,2021-01-02,35.792146,74.982138,4862.0,101.0,4713.0,141.94,291.55,...,3.48,2.40,5.13,1.03,0.0,0.0,0.0,2.395434,5.133114,1.029408
3,Islamabad,Pakistan,2021-01-02,33.665087,73.121219,38020.0,421.0,34617.0,1909.63,3303.93,...,24.67,26.61,43.25,11.48,0.0,0.0,0.0,26.606190,43.253210,11.482480
4,Khyber Pakhtunkhwa,Pakistan,2021-01-02,34.485332,72.091690,59023.0,1661.0,53942.0,73123.81,99366.35,...,463.77,466.71,1027.98,171.97,0.0,0.0,0.0,466.710000,1027.979000,171.967100


In [692]:
df_JHU_IHME.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\JHU_IHME.csv', index = False)

# Merging Weather related Data to JHU_IHME

In [693]:
path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\JHU_IHME.csv'

In [694]:
df_JHU_IHME = pd.read_csv(path)

In [695]:
# df_JHU_IHME.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [696]:
df_JHU_IHME.head(7)

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,inf_mean,inf_upper,...,icu_beds_lower,admis_mean,admis_upper,admis_lower,admis_mean_vax,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax
0,Azad Jammu and Kashmir,Pakistan,2021-01-02,34.027401,73.947253,8303.0,224.0,7691.0,3808.83,6326.40,...,43.54,44.32,87.53,18.54,0.0,0.0,0.0,44.315680,87.531220,18.540050
1,Balochistan,Pakistan,2021-01-02,28.328492,65.898403,18181.0,184.0,17752.0,7796.48,10691.60,...,45.43,51.53,99.96,24.12,0.0,0.0,0.0,51.534680,99.960110,24.121890
2,Gilgit-Baltistan,Pakistan,2021-01-02,35.792146,74.982138,4862.0,101.0,4713.0,141.94,291.55,...,3.48,2.40,5.13,1.03,0.0,0.0,0.0,2.395434,5.133114,1.029408
3,Islamabad,Pakistan,2021-01-02,33.665087,73.121219,38020.0,421.0,34617.0,1909.63,3303.93,...,24.67,26.61,43.25,11.48,0.0,0.0,0.0,26.606190,43.253210,11.482480
4,Khyber Pakhtunkhwa,Pakistan,2021-01-02,34.485332,72.091690,59023.0,1661.0,53942.0,73123.81,99366.35,...,463.77,466.71,1027.98,171.97,0.0,0.0,0.0,466.710000,1027.979000,171.967100
5,Punjab,Pakistan,2021-01-02,30.811346,72.139132,139341.0,4085.0,124125.0,122160.10,165805.40,...,1542.65,1367.06,2409.71,644.18,0.0,0.0,0.0,1367.064000,2409.708000,644.182300
6,Sindh,Pakistan,2021-01-02,26.009446,68.776807,216632.0,3582.0,196134.0,119042.10,150409.20,...,806.62,736.08,1154.62,419.94,0.0,0.0,0.0,736.080700,1154.621000,419.945000


In [697]:
sindh_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Sindh.csv'
punjab_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Punjab.csv'
isb_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\ISB.csv'
bal_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Bal.csv'
ajk_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\AJK.csv'
kpk_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\KPK.csv'
gb_weather_path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\GB.csv'

In [698]:
df_sindh = pd.read_csv(sindh_weather_path)
df_sindh.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_punjab = pd.read_csv(punjab_weather_path)
df_punjab.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_isb = pd.read_csv(isb_weather_path)
df_isb.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_bal = pd.read_csv(bal_weather_path)
df_bal.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_ajk = pd.read_csv(ajk_weather_path)
df_ajk.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_kpk = pd.read_csv(kpk_weather_path)
df_kpk.drop(['Unnamed: 0'], axis = 1, inplace = True)

df_gb = pd.read_csv(gb_weather_path)
df_gb.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [699]:
df_sindh.head(2)

,"January 1, 2020","January 2, 2020","January 3, 2020","January 4, 2020","January 5, 2020","January 6, 2020","January 7, 2020","January 8, 2020","January 9, 2020","January 10, 2020",...,"February 2, 2022","February 3, 2022","February 4, 2022","February 5, 2022","February 6, 2022","February 7, 2022","February 8, 2022","February 9, 2022",Previous 24 hours,"February 10, 2022"
0,"{'Time': '5:00 amWed, Jan 1', 'Temp': '44°F', ...","{'Time': '5:00 amWed, Jan 1', 'Temp': '44°F', ...","{'Time': '2:00 amThu, Jan 2', 'Temp': '50°F', ...","{'Time': '8:00 amFri, Jan 3', 'Temp': '52°F', ...","{'Time': '2:00 amSat, Jan 4', 'Temp': '54°F', ...","{'Time': '8:00 amSun, Jan 5', 'Temp': '57°F', ...","{'Time': '2:00 amMon, Jan 6', 'Temp': '60°F', ...","{'Time': '8:00 amTue, Jan 7', 'Temp': '50°F', ...","{'Time': '5:00 amWed, Jan 8', 'Temp': '53°F', ...","{'Time': '2:00 amThu, Jan 9', 'Temp': '52°F', ...",...,"{'Time': '2:00 amThu, Feb 3', 'Temp': '69°F', ...","{'Time': '2:00 amFri, Feb 4', 'Temp': '61°F', ...","{'Time': '2:00 amSat, Feb 5', 'Temp': '61°F', ...","{'Time': '2:00 amSun, Feb 6', 'Temp': '64°F', ...","{'Time': '5:00 amMon, Feb 7', 'Temp': '63°F', ...","{'Time': '2:00 amTue, Feb 8', 'Temp': '66°F', ...","{'Time': '2:00 amWed, Feb 9', 'Temp': '62°F', ...","{'Time': '2:00 amThu, Feb 10', 'Temp': '62°F',...","{'Time': '8:00 amThu, Feb 10', 'Temp': '55°F',...","{'Time': '8:00 amThu, Feb 10', 'Temp': '55°F',..."
1,"{'Time': '8:00 am', 'Temp': '44°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '44°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '48°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '58°F', 'Weather'...","{'Time': '5:00 am', 'Temp': '50°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '69°F', 'Weather'...","{'Time': '5:00 am', 'Temp': '58°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '55°F', 'Weather'...","{'Time': '8:00 am', 'Temp': '51°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '49°F', 'Weather':...",...,"{'Time': '11:00 am', 'Temp': '72°F', 'Weather'...","{'Time': '5:00 am', 'Temp': '58°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '57°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '59°F', 'Weather':...","{'Time': '11:00 am', 'Temp': '69°F', 'Weather'...","{'Time': '5:00 am', 'Temp': '81°F', 'Weather':...","{'Time': '8:00 am', 'Temp': '58°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '59°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '59°F', 'Weather':...","{'Time': '5:00 am', 'Temp': '59°F', 'Weather':..."


In [700]:
ast.literal_eval(df_sindh['January 1, 2020'][0])

{'Time': '5:00 amWed, Jan 1',
 'Temp': '44°F',
 'Weather': 'Fog.',
 'Wind': '9 mph',
 'Humidity': '↑',
 'Barometer': '86%',
 'Visibility': '30.06 "Hg'}

In [701]:
ast.literal_eval(df_sindh['January 1, 2020'][3])['Weather']

'Partly sunny.'

In [702]:
sindh_final_dict = {}
for i in df_sindh.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_sindh[i])):
        try:
            tTemp += float(ast.literal_eval(df_sindh[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_sindh[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_sindh[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_sindh[i][j])['Visibility'][:5])
            
        except:
            continue
    sindh_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_sindh[i][0])['Weather']}
    

In [703]:
punjab_final_dict = {}
for i in df_punjab.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_punjab[i])):
        try:
            tTemp += float(ast.literal_eval(df_punjab[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_punjab[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_punjab[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_punjab[i][j])['Visibility'][:5])
            
        except:
            continue
    punjab_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_punjab[i][0])['Weather']}
    

In [704]:
isb_final_dict = {}
for i in df_isb.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_isb[i])):
        try:
            tTemp += float(ast.literal_eval(df_isb[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_isb[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_isb[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_isb[i][j])['Visibility'][:5])
            
        except:
            continue
    isb_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_isb[i][0])['Weather']}
    

In [705]:
bal_final_dict = {}
for i in df_bal.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_bal[i])):
        try:
            tTemp += float(ast.literal_eval(df_bal[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_bal[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_bal[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_bal[i][j])['Visibility'][:5])
            
        except:
            continue
    bal_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_bal[i][0])['Weather']}
    

In [706]:
ajk_final_dict = {}
for i in df_ajk.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_ajk[i])):
        try:
            tTemp += float(ast.literal_eval(df_ajk[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_ajk[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_ajk[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_ajk[i][j])['Visibility'][:5])
            
        except:
            continue
    ajk_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_ajk[i][0])['Weather']}
    

In [707]:
kpk_final_dict = {}
for i in df_kpk.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_kpk[i])):
        try:
            tTemp += float(ast.literal_eval(df_kpk[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_kpk[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_kpk[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_kpk[i][j])['Visibility'][:5])
            
        except:
            continue
    kpk_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_kpk[i][0])['Weather']}
    

In [708]:
gb_final_dict = {}
for i in df_gb.columns:
    tTemp = 0
    tWind = 0
    tBarometer = 0
    tVisibility = 0
    for j in range(len(df_gb[i])):
        try:
            tTemp += float(ast.literal_eval(df_gb[i][j])['Temp'][:2])
            tWind += float(ast.literal_eval(df_gb[i][j])['Wind'][:2])
            tBarometer += float(ast.literal_eval(df_gb[i][j])['Barometer'][:2])
            tVisibility += float(ast.literal_eval(df_gb[i][j])['Visibility'][:5])
            
        except:
            continue
    gb_final_dict[i] = {"Temp" : round(tTemp/j, 2), "Wind" : round(tWind/j, 2), "Barometer" : round(tBarometer/j, 2),
                          "Visibility" : round(tVisibility/j, 2), "Weather" : ast.literal_eval(df_gb[i][0])['Weather']}
    

In [709]:
pd.DataFrame(sindh_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\Sindh_Final_Weather.csv')

In [710]:
pd.DataFrame(punjab_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\Punjab_Final_Weather.csv')

In [711]:
pd.DataFrame(isb_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\ISB_Final_Weather.csv')

In [712]:
pd.DataFrame(kpk_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\KPK_Final_Weather.csv')

In [713]:
pd.DataFrame(ajk_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\AJK_Final_Weather.csv')

In [714]:
pd.DataFrame(bal_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\BAL_Final_Weather.csv')

In [715]:
pd.DataFrame(gb_final_dict).T.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\GB_Final_Weather.csv')

In [716]:
dfw_sindh = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\Sindh_Final_Weather.csv')
dfw_sindh["Province"] = "Sindh"
dfw_punjab = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\Punjab_Final_Weather.csv')
dfw_punjab["Province"] = "Punjab"
dfw_ajk = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\AJK_Final_Weather.csv')
dfw_ajk["Province"] = "Azad Jammu and Kashmir"
dfw_kpk = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\KPK_Final_Weather.csv')
dfw_kpk["Province"] = "Khyber Pakhtunkhwa"
dfw_bal = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\BAL_Final_Weather.csv')
dfw_bal["Province"] = "Balochistan"
dfw_gb = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\ISB_Final_Weather.csv')
dfw_gb["Province"] = "Gilgit-Baltistan"
dfw_isb = pd.read_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\Weather Data\\Sindh_Final_Weather.csv')
dfw_isb["Province"] = "Islamabad"
# dfw_sindh.rename(columns={'Unnamed: 0':'Date'}, inplace=True )

In [717]:
dfw_sindh.columns.values[0] = 'Date'
dfw_punjab.columns.values[0] = 'Date'
dfw_ajk.columns.values[0] = 'Date'
dfw_kpk.columns.values[0] = 'Date'
dfw_bal.columns.values[0] = 'Date'
dfw_gb.columns.values[0] = 'Date'
dfw_isb.columns.values[0] = 'Date'

In [718]:
dfw_all_provinces = pd.concat([dfw_sindh, dfw_punjab, dfw_ajk, dfw_kpk, dfw_bal, dfw_gb, dfw_isb])
# dfw_all_provinces.rename(columns={'Unnamed: 0':'Date'}, inplace=True )
dfw_all_provinces[dfw_all_provinces['Date'] == "Previous 24 hours"] = ""

In [719]:
dfw_all_provinces.head()

,Date,Temp,Wind,Barometer,Visibility,Weather,Province
0,"January 1, 2020",43.43,16.43,60.57,25.81,Fog.,Sindh
1,"January 2, 2020",43.43,16.43,60.57,25.81,Fog.,Sindh
2,"January 3, 2020",30.0,9.57,39.86,17.23,Clear.,Sindh
3,"January 4, 2020",42.29,6.14,34.29,17.22,Scattered clouds.,Sindh
4,"January 5, 2020",44.14,0.0,0.0,0.0,Clear.,Sindh


In [720]:
dfw_all_provinces['Date'] = pd.to_datetime(dfw_all_provinces['Date'])

In [721]:
df_JHU_IHME['Date'] = pd.to_datetime(df_JHU_IHME['Date'])

In [722]:
df_JHU_IHME_TD = pd.merge(left=df_JHU_IHME, right=dfw_all_provinces, how='left', left_on=['Date', 'Province'], right_on=['Date', 'Province'])

In [723]:
df_JHU_IHME_TD.head()

,Province,Country_Region,Date,Lat,Long_,Confirmed,Deaths,Recovered,inf_mean,inf_upper,...,admis_upper_vax,admis_lower_vax,admis_mean_unvax,admis_upper_unvax,admis_lower_unvax,Temp,Wind,Barometer,Visibility,Weather
0,Azad Jammu and Kashmir,Pakistan,2021-01-02,34.027401,73.947253,8303.0,224.0,7691.0,3808.83,6326.40,...,0.0,0.0,44.315680,87.531220,18.540050,26.86,1.43,42.71,0.0,Clear.
1,Balochistan,Pakistan,2021-01-02,28.328492,65.898403,18181.0,184.0,17752.0,7796.48,10691.60,...,0.0,0.0,51.534680,99.960110,24.121890,10.43,2.43,8.0,0.0,Clear.
2,Gilgit-Baltistan,Pakistan,2021-01-02,35.792146,74.982138,4862.0,101.0,4713.0,141.94,291.55,...,0.0,0.0,2.395434,5.133114,1.029408,27.86,0.71,12.14,4.3,Fog.
3,Islamabad,Pakistan,2021-01-02,33.665087,73.121219,38020.0,421.0,34617.0,1909.63,3303.93,...,0.0,0.0,26.606190,43.253210,11.482480,32.57,4.0,28.0,17.15,Clear.
4,Khyber Pakhtunkhwa,Pakistan,2021-01-02,34.485332,72.091690,59023.0,1661.0,53942.0,73123.81,99366.35,...,0.0,0.0,466.710000,1027.979000,171.967100,44.34,3.86,36.73,23.24,Passing clouds.


In [724]:
df_JHU_IHME_TD.columns

Index(['Province', 'Country_Region', 'Date', 'Lat', 'Long_', 'Confirmed',
       'Deaths', 'Recovered', 'inf_mean', 'inf_upper', 'inf_lower',
       'seir_cumulative_mean', 'seir_cumulative_lower',
       'seir_cumulative_upper', 'inf_cuml_mean', 'inf_cuml_lower',
       'inf_cuml_upper', 'seir_daily_mean', 'seir_daily_lower',
       'seir_daily_upper', 'seir_daily_unscaled_mean',
       'seir_daily_unscaled_lower', 'seir_daily_unscaled_upper',
       'seir_cumulative_unscaled_mean', 'seir_cumulative_unscaled_lower',
       'seir_cumulative_unscaled_upper', 'cases_mean', 'cases_lower',
       'cases_upper', 'inf_cuml_mean_vax', 'inf_cuml_lower_vax',
       'inf_cuml_upper_vax', 'inf_cuml_mean_unvax', 'inf_cuml_lower_unvax',
       'inf_cuml_upper_unvax', 'inf_mean_vax', 'inf_lower_vax',
       'inf_upper_vax', 'inf_mean_unvax', 'inf_lower_unvax', 'inf_upper_unvax',
       'population', 'mobility_mean', 'mobility_obs', 'testing_obs',
       'pneumonia_mean', 'pneumonia_obs', 'mask_use_m

In [725]:
pd.options.mode.chained_assignment = None  # default='warn'
columns = ['Confirmed',
       'Deaths', 'Recovered', 'inf_mean', 'inf_upper', 'inf_lower',
       'seir_cumulative_mean', 'seir_cumulative_lower',
       'seir_cumulative_upper', 'inf_cuml_mean', 'inf_cuml_lower',
       'inf_cuml_upper', 'seir_daily_mean', 'seir_daily_lower',
       'seir_daily_upper', 'seir_daily_unscaled_mean',
       'seir_daily_unscaled_lower', 'seir_daily_unscaled_upper',
       'seir_cumulative_unscaled_mean', 'seir_cumulative_unscaled_lower',
       'seir_cumulative_unscaled_upper', 'cases_mean', 'cases_lower',
       'cases_upper', 'inf_cuml_mean_vax', 'inf_cuml_lower_vax',
       'inf_cuml_upper_vax', 'inf_cuml_mean_unvax', 'inf_cuml_lower_unvax',
       'inf_cuml_upper_unvax', 'inf_mean_vax', 'inf_lower_vax',
       'inf_upper_vax', 'inf_mean_unvax', 'inf_lower_unvax', 'inf_upper_unvax',
       'population', 'mobility_mean', 'mobility_obs', 'testing_obs',
       'pneumonia_mean', 'pneumonia_obs', 'mask_use_mean', 'mask_use_obs',
       'hospital_beds_mean', 'hospital_beds_upper', 'hospital_beds_lower',
       'icu_beds_mean', 'icu_beds_upper', 'icu_beds_lower', 'admis_mean',
       'admis_upper', 'admis_lower', 'admis_mean_vax', 'admis_upper_vax',
       'admis_lower_vax', 'admis_mean_unvax', 'admis_upper_unvax',
       'admis_lower_unvax', "Temp", "Wind", "Barometer", "Visibility"]

for i in range(len(df_JHU_IHME_TD)):
  for j in columns:
    if df_JHU_IHME_TD[j][i] < 0:
        df_JHU_IHME_TD[j][i] = abs(df_JHU_IHME_TD[j][i])

In [726]:
df_JHU_IHME_TD.to_csv('C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Master\\FYP-I\\Data Collection\\CollectedData\\JHU_IHME_TD.csv', index = False)

In [637]:
# Done till here

# Preparing Vaccination Data

In [ ]:
path = "C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\Data (CSSE) at Johns Hopkins University\\COVID-19\\vaccinations_data\\vaccinations.csv"

In [ ]:
df_vaccinations = pd.read_csv(path)

In [ ]:
df_vac_pak = df_vaccinations[df_vaccinations['location'] == "Pakistan"]
df_vac_pak.head()

In [ ]:
df_vac_pak = df_vac_pak[['date', 'daily_vaccinations', 'daily_vaccinations_per_million']]
df_vac_pak = df_vac_pak.dropna(subset=['daily_vaccinations', 'daily_vaccinations_per_million'])
df_vac_pak.head()

# Merging Vaccination Data to JHU_IHME

In [ ]:
path = 'C:\\Users\\OvaizAli\\Desktop\\FYP-2021\\Working Directory\\JHU_IHME.csv'
df_JHU_IHME = pd.read_csv(path)

In [ ]:
df_JHU_IHME_Vac = pd.merge(left=df_JHU_IHME, right=df_vac_pak, how='left', left_on=['Date', 'Province'], right_on=['Date', 'Province'])